In [1]:

import duckdb 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from src.feature_engineering import feature_engineering_lag , feature_engineering_delta


%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb

In [2]:
data_path = "data/"
data_file = "competencia_01.csv"
df= pd.read_csv(data_path + data_file)

C:\Users\chris\AppData\Local\Temp\ipykernel_20832\4193464990.py:3: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(data_path + data_file)


#

In [3]:
cat_cols =[]
num_cols=[]
for c in df.columns:
    if (df[c].nunique() <= 5):
        cat_cols.append(c)
    else:
        num_cols.append(c)

In [4]:
lista_t=[c for c in list(map(lambda x : x if x[0]=='t' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_c=[c for c in list(map(lambda x : x if x[0]=='c' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_m=[c for c in list(map(lambda x : x if x[0]=='m' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_r=[c for c in df.columns if c not in (lista_t + lista_c + lista_m + cat_cols)]
print(len(lista_t) , len(lista_c) , len(lista_m) , len(lista_r) , len(cat_cols))
print(len(lista_t) + len(lista_c) + len(lista_m) +len(lista_r) + len(cat_cols))

0 48 45 42 20
155


In [5]:
col_drops=["cliente_edad","numero_de_cliente","cliente_antiguedad"]

In [6]:
cols_transf_previo=lista_m + lista_c+ lista_r
cols_trans = [c for c in cols_transf_previo if c not in col_drops]
cols_trans

['mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'mcuentas_saldo',
 'mautoservicio',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'mprestamos_personales',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'minversion1_pesos',
 'minversion1_dolares',
 'minversion2',
 'mpayroll',
 'mpayroll2',
 'mcuenta_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'mpagodeservicios',
 'mpagomiscuentas',
 'mcajeros_propios_descuentos',
 'mtarjeta_visa_descuentos',
 'mtarjeta_master_descuentos',
 'mcomisiones_mantenimiento',
 'mcomisiones_otras',
 'mforex_buy',
 'mforex_sell',
 'mtransferencias_recibidas',
 'mtransferencias_emitidas',
 'mextraccion_autoservicio',
 'mcheques_depositados',
 'mcheques_emitidos',
 'mcheque

In [8]:
df_2 = feature_engineering_lag(df ,cols_trans , 2)
df_3 = feature_engineering_delta(df_2 ,cols_trans , 2)

mrentabilidad
mrentabilidad_annual
mcomisiones
mactivos_margen
mpasivos_margen
mcuenta_corriente_adicional
mcuenta_corriente
mcaja_ahorro
mcaja_ahorro_adicional
mcaja_ahorro_dolares
mcuentas_saldo
mautoservicio
mtarjeta_visa_consumo
mtarjeta_master_consumo
mprestamos_personales
mprestamos_prendarios
mprestamos_hipotecarios
mplazo_fijo_dolares
mplazo_fijo_pesos
minversion1_pesos
minversion1_dolares
minversion2
mpayroll
mpayroll2
mcuenta_debitos_automaticos
mttarjeta_visa_debitos_automaticos
mttarjeta_master_debitos_automaticos
mpagodeservicios
mpagomiscuentas
mcajeros_propios_descuentos
mtarjeta_visa_descuentos
mtarjeta_master_descuentos
mcomisiones_mantenimiento
mcomisiones_otras
mforex_buy
mforex_sell
mtransferencias_recibidas
mtransferencias_emitidas
mextraccion_autoservicio
mcheques_depositados
mcheques_emitidos
mcheques_depositados_rechazados
mcheques_emitidos_rechazados
matm
matm_other
cproductos
ccaja_ahorro
ctarjeta_debito
ctarjeta_debito_transacciones
ctarjeta_visa_transaccione

In [ ]:
%%sql 
SELECT numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1 , mrentabilidad_annual_lag_2 , delta_1_mrentabilidad_annual , delta_2_mrentabilidad_annual
FROM df_3
ORDER BY (numero_de_cliente,foto_mes )

,numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1,mrentabilidad_annual_lag_2,delta_1_mrentabilidad_annual,delta_2_mrentabilidad_annual
0,249221323,202101,15691.10,NaN,NaN,NaN,NaN
1,249221323,202102,19103.66,15691.10,NaN,3412.56,NaN
2,249221323,202103,24020.21,19103.66,15691.10,4916.55,8329.11
3,249221323,202104,29189.81,24020.21,19103.66,5169.60,10086.15
4,249221323,202105,31505.78,29189.81,24020.21,2315.97,7485.57
...,...,...,...,...,...,...,...
978434,1598368433,202106,4123.09,NaN,NaN,NaN,NaN
978435,1598419415,202106,-2.26,NaN,NaN,NaN,NaN
978436,1598425905,202106,5.92,NaN,NaN,NaN,NaN
978437,1598470389,202106,244.96,NaN,NaN,NaN,NaN


In [9]:
%%sql 
SELECT numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1 , mrentabilidad_annual_lag_2 , delta_1_mrentabilidad_annual , delta_2_mrentabilidad_annual
FROM df_3
ORDER BY (numero_de_cliente,foto_mes )

,numero_de_cliente,foto_mes,mrentabilidad_annual,mrentabilidad_annual_lag_1,mrentabilidad_annual_lag_2,delta_1_mrentabilidad_annual,delta_2_mrentabilidad_annual
0,249221323,202101,15691.10,NaN,NaN,NaN,NaN
1,249221323,202102,19103.66,15691.10,NaN,3412.56,NaN
2,249221323,202103,24020.21,19103.66,15691.10,4916.55,8329.11
3,249221323,202104,29189.81,24020.21,19103.66,5169.60,10086.15
4,249221323,202105,31505.78,29189.81,24020.21,2315.97,7485.57
...,...,...,...,...,...,...,...
978434,1598368433,202106,4123.09,NaN,NaN,NaN,NaN
978435,1598419415,202106,-2.26,NaN,NaN,NaN,NaN
978436,1598425905,202106,5.92,NaN,NaN,NaN,NaN
978437,1598470389,202106,244.96,NaN,NaN,NaN,NaN


In [13]:
df_3

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,delta_1_Visa_fechaalta,delta_2_Visa_fechaalta,delta_1_Visa_mconsumototal,delta_2_Visa_mconsumototal,delta_1_Visa_cconsumos,delta_2_Visa_cconsumos,delta_1_Visa_cadelantosefectivo,delta_2_Visa_cadelantosefectivo,delta_1_Visa_mpagominimo,delta_2_Visa_mpagominimo
0,249320580,202101,1,0,0,63,279,508.80,17140.27,896.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249320580,202102,1,0,0,63,280,2494.79,18472.46,2015.94,...,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2,249320580,202103,1,0,0,63,281,3207.66,19030.88,2519.75,...,31.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
3,249320580,202104,1,0,0,64,282,3122.15,19147.05,2472.63,...,30.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
4,249320580,202105,1,0,0,64,283,3455.86,20443.48,2520.36,...,31.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,462675490,202103,1,0,0,47,128,1233.24,18914.15,3543.85,...,31.0,59.0,15095.31,-486.35,7.0,5.0,0.0,0.0,-351.90,-93.84
978435,462675490,202104,1,0,0,47,129,-1120.49,15020.31,3531.58,...,30.0,61.0,1292.20,16387.51,-1.0,6.0,0.0,0.0,-58.65,-410.55
978436,462675490,202105,1,0,0,47,130,1054.73,13273.80,4140.06,...,31.0,61.0,-6611.25,-5319.05,12.0,11.0,0.0,0.0,703.80,645.15
978437,462675490,202106,1,0,0,47,131,-3003.60,9288.37,3139.78,...,30.0,61.0,1209.14,-5402.11,-6.0,6.0,0.0,0.0,-844.56,-140.76


In [15]:
mes =202104

f = df_3["foto_mes"] == mes
df = df_3.loc[f]